# Stats Processing

In [ ]:
import pandas as pd
import os, json

DATASET = "./GarminRawData"

## Load data

In [ ]:
files = []
for dirName in os.listdir( DATASET ):

    basedir = os.path.join( DATASET, dirName )

    if os.path.isdir( basedir ):
        for fName in os.listdir( basedir ):
            if fName.endswith( "-stat.json" ):
                fName = os.path.join( basedir, fName )
                print( f"""Found file: {fName}""" )
                files.append( fName )

print( files )

In [ ]:
data = []

for fName in files:
    print(fName)
    try:
        f=json.load( open( fName, 'rt') )
        data.append(f)
    except json.JSONDecodeError:
        print(f"{fName} vuoto")

In [ ]:
stat_df=pd.DataFrame.from_records(data)
pd.DataFrame(stat_df).to_csv( "GarminStats.csv", index=False )

In [ ]:
stat_df['day'] = pd.to_datetime(stat_df['t_start'].str[:10])
stat_df['t_start'] = pd.to_datetime(stat_df['t_start'])
stat_df['t_stop'] = pd.to_datetime(stat_df['t_stop'])
stat_df['time_created'] = pd.to_datetime(stat_df['time_created'])

In [ ]:
stat_df

## Filtering
### By duration

In [ ]:
tresh_sec = 1800
stat_df.loc[stat_df['duration_sec'] > tresh_sec].groupby('day').count()

In [ ]:
stat_df=stat_df.loc[stat_df['duration_sec'] > tresh_sec]
stat_df.shape

### by t_start

In [ ]:
stat_df=stat_df.loc[(stat_df['t_start'].dt.hour >= 8) & (stat_df['t_start'].dt.hour <= 11)]
print(stat_df.shape)
stat_df.groupby('day').count()

In [ ]:
stat_df.loc[(stat_df['day'] == '2024-05-23') &(stat_df['duration_sec'] > tresh_sec)]

In [ ]:
stat_df.to_csv('stats.csv')

In [ ]:
stat_df[['serial_number', 'day', 'duration_sec']].pivot_table(columns='serial_number', index='day', aggfunc='sum').fillna(0).to_csv('duration_day_watch.csv')